# Day 20: Pulse Propagation
With your help, the Elves manage to find the right parts and fix all of the machines. Now, they just need to send the command to boot up the machines and get the sand flowing again.

The machines are far apart and wired together with long cables. The cables don't connect to the machines directly, but rather to communication modules attached to the machines that perform various initialization tasks and also act as communication relays.

Modules communicate using pulses. Each pulse is either a high pulse or a low pulse. When a module sends a pulse, it sends that type of pulse to each module in its list of destination modules.

There are several different types of modules:

Flip-flop modules (prefix %) are either on or off; they are initially off. If a flip-flop module receives a high pulse, it is ignored and nothing happens. However, if a flip-flop module receives a low pulse, it flips between on and off. If it was off, it turns on and sends a high pulse. If it was on, it turns off and sends a low pulse.

Conjunction modules (prefix &) remember the type of the most recent pulse received from each of their connected input modules; they initially default to remembering a low pulse for each input. When a pulse is received, the conjunction module first updates its memory for that input. Then, if it remembers high pulses for all inputs, it sends a low pulse; otherwise, it sends a high pulse.

There is a single broadcast module (named broadcaster). When it receives a pulse, it sends the same pulse to all of its destination modules.

Here at Desert Machine Headquarters, there is a module with a single button on it called, aptly, the button module. When you push the button, a single low pulse is sent directly to the broadcaster module.

After pushing the button, you must wait until all pulses have been delivered and fully handled before pushing it again. Never push the button if modules are still processing pulses.

Pulses are always processed in the order they are sent. So, if a pulse is sent to modules a, b, and c, and then module a processes its pulse and sends more pulses, the pulses sent to modules b and c would have to be handled first.

The module configuration (your puzzle input) lists each module. The name of the module is preceded by a symbol identifying its type, if any. The name is then followed by an arrow and a list of its destination modules. For example:
```
broadcaster -> a, b, c
%a -> b
%b -> c
%c -> inv
&inv -> a
```
In this module configuration, the broadcaster has three destination modules named a, b, and c. Each of these modules is a flip-flop module (as indicated by the % prefix). a outputs to b which outputs to c which outputs to another module named inv. inv is a conjunction module (as indicated by the & prefix) which, because it has only one input, acts like an inverter (it sends the opposite of the pulse type it receives); it outputs to a.

By pushing the button once, the following pulses are sent:
```
button -low-> broadcaster
broadcaster -low-> a
broadcaster -low-> b
broadcaster -low-> c
a -high-> b
b -high-> c
c -high-> inv
inv -low-> a
a -low-> b
b -low-> c
c -low-> inv
inv -high-> a
```
After this sequence, the flip-flop modules all end up off, so pushing the button again repeats the same sequence.

Here's a more interesting example:
```
broadcaster -> a
%a -> inv, con
&inv -> b
%b -> con
&con -> output
```
This module configuration includes the broadcaster, two flip-flops (named a and b), a single-input conjunction module (inv), a multi-input conjunction module (con), and an untyped module named output (for testing purposes). The multi-input conjunction module con watches the two flip-flop modules and, if they're both on, sends a low pulse to the output module.

Here's what happens if you push the button once:
```
button -low-> broadcaster
broadcaster -low-> a
a -high-> inv
a -high-> con
inv -low-> b
con -high-> output
b -high-> con
con -low-> output
```
Both flip-flops turn on and a low pulse is sent to output! However, now that both flip-flops are on and con remembers a high pulse from each of its two inputs, pushing the button a second time does something different:
```
button -low-> broadcaster
broadcaster -low-> a
a -low-> inv
a -low-> con
inv -high-> b
con -high-> output
```
Flip-flop a turns off! Now, con remembers a low pulse from module a, and so it sends only a high pulse to output.

Push the button a third time:
```
button -low-> broadcaster
broadcaster -low-> a
a -high-> inv
a -high-> con
inv -low-> b
con -low-> output
b -low-> con
con -high-> output
```
This time, flip-flop a turns on, then flip-flop b turns off. However, before b can turn off, the pulse sent to con is handled first, so it briefly remembers all high pulses for its inputs and sends a low pulse to output. After that, flip-flop b turns off, which causes con to update its state and send a high pulse to output.

Finally, with a on and b off, push the button a fourth time:
```
button -low-> broadcaster
broadcaster -low-> a
a -low-> inv
a -low-> con
inv -high-> b
con -high-> output
```
This completes the cycle: a turns off, causing con to remember only low pulses and restoring all modules to their original states.

To get the cables warmed up, the Elves have pushed the button 1000 times. How many pulses got sent as a result (including the pulses sent by the button itself)?

In the first example, the same thing happens every time the button is pushed: 8 low pulses and 4 high pulses are sent. So, after pushing the button 1000 times, 8000 low pulses and 4000 high pulses are sent. Multiplying these together gives 32000000.

In the second example, after pushing the button 1000 times, 4250 low pulses and 2750 high pulses are sent. Multiplying these together gives 11687500.

Consult your module configuration; determine the number of low pulses and high pulses that would be sent after pushing the button 1000 times, waiting for all pulses to be fully handled after each push of the button. What do you get if you multiply the total number of low pulses sent by the total number of high pulses sent?

In [20]:
from collections import defaultdict
def revertStrength(strength):
    return 'low' if strength == 'high' else 'high'

def result(inputfile):
    types = {}
    destinations = {}
    currentStates = defaultdict(lambda: {})
    with open(inputfile) as f:
        for line in f:
            typeAndName, destList = line.strip().split(' -> ')
            if typeAndName[0] == '%':
                types[typeAndName[1:]] = typeAndName[0]
                currentStates[typeAndName[1:]] = 'low'
                destinations[typeAndName[1:]] = destList.split(', ')
            elif typeAndName[0] == '&':
                types[typeAndName[1:]] = typeAndName[0]
                destinations[typeAndName[1:]] = destList.split(', ')
            else:
                destinations[typeAndName] = destList.split(', ')
    for key in types.keys():
        for dest in destinations[key]:
            if dest in types.keys() and types[dest] == '&':
                currentStates[dest][key] = 'low'

    lows = 0
    highs = 0

    for _ in range(1000):
        pulseQueue = [['broadcaster', 'low','']]
        while len(pulseQueue) != 0:
            name, strength, sender = pulseQueue.pop(0)
            if strength == 'low':
                lows += 1
            else:
                highs += 1
            if name == 'broadcaster':
                for dest in destinations[name]:
                    pulseQueue.append([dest, strength, name])
            elif name not in types.keys():
                continue
            elif types[name] == '%' and strength == 'low':
                reverse  = revertStrength(currentStates[name])
                currentStates[name] = reverse
                for dest in destinations[name]:
                    pulseQueue.append([dest, reverse, name])
            elif types[name] == '&':
                currentStates[name][sender] = strength
                if all([x == 'high' for x in currentStates[name].values()]):
                    for dest in destinations[name]:
                        pulseQueue.append([dest, 'low', name])
                else:
                    for dest in destinations[name]:
                        pulseQueue.append([dest, 'high', name])

    return lows, highs, lows*highs 


In [21]:
result("example-input.txt")

(8000, 4000, 32000000)

In [22]:
result("example-input-2.txt")

(4250, 2750, 11687500)

In [23]:
result("input.txt")

(18771, 46872, 879834312)

### Part Two
The final machine responsible for moving the sand down to Island Island has a module attached named rx. The machine turns on when a single low pulse is sent to rx.

Reset all modules to their default states. Waiting for all pulses to be fully handled after each button press, what is the fewest number of button presses required to deliver a single low pulse to the module named rx?

In [19]:
from pyvis.network import Network
import networkx as nx

def generateGraphHtml(inputfile):
    destinations = {}
    with open(inputfile) as f:
        for line in f:
            typeAndName, destList = line.strip().split(' -> ')
            if typeAndName[0] == '%':
                destinations[typeAndName[1:]] = destList.split(', ')
            elif typeAndName[0] == '&':
                destinations[typeAndName[1:]] = destList.split(', ')
            else:
                destinations[typeAndName] = destList.split(', ')
    
    net = Network(directed=True)

    G = nx.from_dict_of_lists(destinations, create_using=nx.DiGraph)
    net.from_nx(G)
    net.write_html("graph.html")

In [13]:
generateGraphHtml("input.txt")

In [49]:
def result2(inputfile, target):
    types = {}
    destinations = {}
    currentStates = defaultdict(lambda: {})
    inputs = defaultdict(lambda: [])
    with open(inputfile) as f:
        for line in f:
            typeAndName, destList = line.strip().split(' -> ')
            if typeAndName[0] == '%':
                types[typeAndName[1:]] = typeAndName[0]
                currentStates[typeAndName[1:]] = 'low'
                destinations[typeAndName[1:]] = destList.split(', ')
            elif typeAndName[0] == '&':
                types[typeAndName[1:]] = typeAndName[0]
                destinations[typeAndName[1:]] = destList.split(', ')
            else:
                destinations[typeAndName] = destList.split(', ')
    for key in destinations.keys():
        bla = inputs[key]
        for dest in destinations[key]:
            inputs[dest].append(key)
            if dest in types.keys() and types[dest] == '&':
                currentStates[dest][key] = 'low'
    
    
    ts = []
    t = 1
    while len(ts) < 10:
        pulseQueue = [['broadcaster', 'low','']]
        while len(pulseQueue) != 0:
            name, strength, sender = pulseQueue.pop(0)
            if name == target and strength == 'low':
                ts.append(t)
            if name == 'broadcaster':
                for dest in destinations[name]:
                    pulseQueue.append([dest, strength, name])
            elif name not in types.keys():
                continue
            elif types[name] == '%' and strength == 'low':
                reverse  = revertStrength(currentStates[name])
                currentStates[name] = reverse
                for dest in destinations[name]:
                    pulseQueue.append([dest, reverse, name])
            elif types[name] == '&':
                currentStates[name][sender] = strength
                if all([x == 'high' for x in currentStates[name].values()]):
                    for dest in destinations[name]:
                        pulseQueue.append([dest, 'low', name])
                else:
                    for dest in destinations[name]:
                        pulseQueue.append([dest, 'high', name])
        t += 1
    for i in range(9):
        print(ts[i+1] - ts[i])
    return ts

In [50]:
result2('input.txt', 'mr')

3907
3907
3907
3907
3907
3907
3907
3907
3907


[3907, 7814, 11721, 15628, 19535, 23442, 27349, 31256, 35163, 39070]

In [51]:
result2('input.txt', 'kk')

3931
3931
3931
3931
3931
3931
3931
3931
3931


[3931, 7862, 11793, 15724, 19655, 23586, 27517, 31448, 35379, 39310]

In [52]:
result2('input.txt', 'bb')

3967
3967
3967
3967
3967
3967
3967
3967
3967


[3967, 7934, 11901, 15868, 19835, 23802, 27769, 31736, 35703, 39670]

In [53]:
result2('input.txt', 'gl')

3989
3989
3989
3989
3989
3989
3989
3989
3989


[3989, 7978, 11967, 15956, 19945, 23934, 27923, 31912, 35901, 39890]

In [54]:
from math import lcm

lcm(3989, 3967, 3931, 3907)

243037165713371